### Calculate total split of wind in Europe by case

In [32]:
import pandas as pd
from constants import OW_TYPES, OFFSHORE_AREAS

CASES = ["Base", "NOgrid", "NoHubs", "NoHubsNOgrid", "OnlyCentral", "OnlyEU", "OnlyNorth", "CapGrounded/CapGroundedN30"]
DF_BY_CASES = dict({})

for case in CASES:
    _df = pd.read_csv(f'Results/{case}/results_output_gen.csv')
    # Filter by tech
    _df = _df[_df["GeneratorType"].isin(OW_TYPES)]
    DF_BY_CASES[case] = _df
DF_BY_CASES

{'Base':           Node         GeneratorType     Period  genInvCap_MW  \
 3296   Belgium  Windoffshoregrounded  2020-2025      0.000000   
 3297   Belgium  Windoffshoregrounded  2025-2030      0.000000   
 3298   Belgium  Windoffshoregrounded  2030-2035      0.000000   
 3299   Belgium  Windoffshoregrounded  2035-2040      0.000000   
 3300   Belgium  Windoffshoregrounded  2040-2045      0.000000   
 ...        ...                   ...        ...           ...   
 5899  SørvestD  Windoffshorefloating  2035-2040      0.000058   
 5900  SørvestD  Windoffshorefloating  2040-2045      0.000224   
 5901  SørvestD  Windoffshorefloating  2045-2050      0.000415   
 5902  SørvestD  Windoffshorefloating  2050-2055      0.000010   
 5903  SørvestD  Windoffshorefloating  2055-2060      0.000745   
 
       genInstalledCap_MW  genExpectedCapacityFactor  \
 3296            0.000000                   0.000000   
 3297            0.000000                   0.000000   
 3298            0.000000     

Filter by case and period

In [35]:
FILTER_CASE = "Base"
FILTER_PERIOD = "2045-2050"

filtered_df = DF_BY_CASES[FILTER_CASE]
filtered_df = filtered_df[filtered_df["Period"] == FILTER_PERIOD]
# Threshold of 1 MW to exclude areas without capacity
filtered_df = filtered_df[filtered_df["genInstalledCap_MW"] > 1]
# MW to GW 
filtered_df["genInvCap_GW"] = round(filtered_df["genInvCap_MW"] / 1000)
filtered_df["genInstalledCap_GW"] = round(filtered_df["genInstalledCap_MW"] / 1000)
filtered_df = filtered_df.sort_values("genInstalledCap_GW", ascending=False).drop(columns=["genInvCap_MW", "genInstalledCap_MW"]).reset_index(drop=True)
filtered_df["genExpectedAnnualProduction_GWh"] = round(filtered_df["genExpectedAnnualProduction_GWh"])
filtered_df["genExpectedCapacityFactor"] = round(filtered_df["genExpectedCapacityFactor"], 2)
# Change column order
filtered_df = filtered_df[["Node", "Period", "GeneratorType", "genInstalledCap_GW", "genExpectedAnnualProduction_GWh", "genExpectedCapacityFactor", "genInvCap_GW", "DiscountedInvestmentCost_Euro"]]

# For Europe total
print(f"OW capacity capacity total in Europe {FILTER_PERIOD.split("-")[1]}: {round(filtered_df["genInstalledCap_GW"].sum())}GW")
print(f"OW capacity floating in Europe {FILTER_PERIOD.split("-")[1]}: {round(filtered_df[filtered_df["GeneratorType"] == "Windoffshorefloating"]["genInstalledCap_GW"].sum())}GW")
print(f"OW capacity grounded in Europe {FILTER_PERIOD.split("-")[1]}: {round(filtered_df[filtered_df["GeneratorType"] == "Windoffshoregrounded"]["genInstalledCap_GW"].sum())}GW\n")

# In North Sea
filtered_df_NS = filtered_df[filtered_df["Node"].isin(OFFSHORE_AREAS)]
print(f"OW capacity capacity total in NS {FILTER_PERIOD.split("-")[1]}: {round(filtered_df_NS["genInstalledCap_GW"].sum())}GW")
print(f"OW capacity floating in NS {FILTER_PERIOD.split("-")[1]}: {round(filtered_df_NS[filtered_df_NS["GeneratorType"] == "Windoffshorefloating"]["genInstalledCap_GW"].sum())}GW")
print(f"OW capacity grounded in NS {FILTER_PERIOD.split("-")[1]}: {round(filtered_df_NS[filtered_df_NS["GeneratorType"] == "Windoffshoregrounded"]["genInstalledCap_GW"].sum())}GW\n")

# Rest of Europe
filtered_df_rest = filtered_df[~filtered_df["Node"].isin(OFFSHORE_AREAS)]
print(f"OW capacity capacity total in rest of Europe {FILTER_PERIOD.split("-")[1]}: {round(filtered_df_rest["genInstalledCap_GW"].sum())}GW")
print(f"OW capacity floating in rest of Europe {FILTER_PERIOD.split("-")[1]}: {round(filtered_df_rest[filtered_df_rest["GeneratorType"] == "Windoffshorefloating"]["genInstalledCap_GW"].sum())}GW")
print(f"OW capacity grounded in rest of Europe {FILTER_PERIOD.split("-")[1]}: {round(filtered_df_rest[filtered_df_rest["GeneratorType"] == "Windoffshoregrounded"]["genInstalledCap_GW"].sum())}GW\n")

filtered_df_rest

OW capacity capacity total in Europe 2050: 303GW
OW capacity floating in Europe 2050: 51GW
OW capacity grounded in Europe 2050: 252GW

OW capacity capacity total in NS 2050: 223GW
OW capacity floating in NS 2050: 51GW
OW capacity grounded in NS 2050: 172GW

OW capacity capacity total in rest of Europe 2050: 80GW
OW capacity floating in rest of Europe 2050: 0GW
OW capacity grounded in rest of Europe 2050: 80GW



,Node,Period,GeneratorType,genInstalledCap_GW,genExpectedAnnualProduction_GWh,genExpectedCapacityFactor,genInvCap_GW,DiscountedInvestmentCost_Euro
0,France,2045-2050,Windoffshoregrounded,58.0,131642.0,0.26,0.0,5.299331e+01
3,Sweden,2045-2050,Windoffshoregrounded,20.0,81371.0,0.47,0.0,6.223080e+01
26,Germany,2045-2050,Windoffshoregrounded,2.0,6818.0,0.34,2.0,1.206588e+09
